In [3]:
import sys
sys.path.append('/cluster/sj1/bb_opt/scripts')

In [4]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collect_stats as cs

%matplotlib inline

In [5]:
import collect_stats as cs

In [6]:
exp_folder = '/cluster/sj1/bb_opt/experiments/imdbwiki'
batches = [20, 50, 200]
num_acks = 50

In [7]:
filenames = ['wiki']

In [8]:
def sigmoid(x, exp=np.exp):
  return 1.0 / (1.0 + exp(-x))

1. ensemble16 - verify paper results

In [9]:
arrs = {}

In [97]:
to_read = {
    #'ensemble18/o_max_std_start_3k_reg_': [None, 'normal'],
    'ensemble18/o_max_std_start_3k_reg_maxinvar_g00020510204080_': [None, 'MTD_big'],
    #'ensemble18/o_max_std_start_3k_reg_uinput_maxvar_g000.10.50102_': [None, 'MOD_small'],
    #'ensemble18/o_max_std_start_3k_reg_uinput_maxvar_g00020510204080_': [None, 'MOD_big'],
    #'ensemble18/o_max_std_start_3k_reg_nc_g00020510204080_': [None, 'NC_big'],
    #'ensemble18/o_max_std_start_3k_reg_nc_g000.10.50102_': [None, 'NC_small'],
    #'ensemble18/o_max_std_start_3k_reg_adve_': [None, 'adve'],
    #'ensemble18/o_max_std_start_3k_reg_id_maxvar_g00020510204080_': [None, 'idMOD_big'],
    
}


In [98]:
for k in to_read:
    arrs[k] = to_read[k]
for experiment in to_read:
    print('reading', experiment)
    arrs[experiment][0] = cs.get_data(exp_folder, experiment, batches, num_samples=20)

reading ensemble18/o_max_std_start_3k_reg_maxinvar_g00020510204080_
reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9
reading sample 10
reading sample 11
reading sample 12
reading sample 13
reading sample 14
reading sample 15
reading sample 16
reading sample 17
reading sample 18
reading sample 19
reading sample 20


In [107]:
to_eval = [
    'ensemble18/o_max_std_start_3k_reg_',
    'ensemble18/o_max_std_start_3k_reg_maxinvar_g00020510204080_',
    'ensemble18/o_max_std_start_3k_reg_uinput_maxvar_g000.10.50102_',
    'ensemble18/o_max_std_start_3k_reg_uinput_maxvar_g00020510204080_',
    'ensemble18/o_max_std_start_3k_reg_nc_g000.10.50102_',
    'ensemble18/o_max_std_start_3k_reg_nc_g00020510204080_',
    'ensemble18/o_max_std_start_3k_reg_adve_',
    'ensemble18/o_max_std_start_3k_reg_id_maxvar_g00020510204080_',
]

In [13]:
test_fn = lambda bs, prop: lambda x, filename : x[bs + 'test_pred_stats'][prop]
bs_test_rmse_fn = lambda x, filename : x['test_pred_stats']['baseline_rmse']
ood_fn = lambda bs, prop: lambda x, filename : x[bs + 'ood_pred_stats'][prop]
bs_ood_rmse_fn = lambda x, filename : x['test_pred_stats']['baseline_rmse']
test_log_fn = lambda prop, prop2: lambda x, filename : x['logging'][1][prop][prop2]

In [100]:
dist = "test"
dist = "ood"
prop = "log_prob"
#prop = "rmse"
#prop = "pred_std"
#prop = 'rmse_std_corr'
bs = "zero_gamma_"
#bs = "invar_gamma_"

#fn = [globals()[dist+"_" + prop + "_fn"](""),globals()[dist + "_" + prop + "_fn"](bs)]
fn = [globals()[dist+"_fn"]("", prop), globals()[dist+"_fn"](bs, prop)]
cs.prop_test2(50, 
             filenames,
             fn, 
             arrs,
             to_eval[1],
             0,
             pval_threshold=1,
             paired_test=True,
            )

comparing <lambda> <lambda>
wiki 0.41171424753134733 -0.13962985426187516 -0.13049787059426307 1 	 (std: 0.03075 0.04777, #n: 20 20)
combined pval: nan vs 0.41171
count: 1/1


In [110]:
cs.prop_test(50, 
             filenames,
             test_fn("", "log_prob"), 
             arrs, 
             [to_eval[5], to_eval[3]],
             0,
             pval_threshold=1,
             paired_test=False,
            )

comparing NC_big MTD_big
wiki 0.763956068023512 -0.328290057182312 -0.3305874511599541 0 	 (std: 0.02135 0.02530, #n: 20 20)
combined pval: 0.76396 vs nan
count: 0/1


In [ ]:
cs.plot_data_vs_ack_iter(
    50,
    filenames,
    'avg_seeds',
    'rmse',
    test_rmse_fn,
    20,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
cs.plot_data_vs_ack_iter(
    20,
    filenames,
    'avg_seeds',
    'log_prob',
    lambda x, filename: x['best_gamma'],
    30,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
cs.plot_data_vs_ack_iter(
    50,
    filenames,
    'avg_seeds',
    'nll',
    lambda x, filename : x['logging'][1]['best']['nll'],
    20,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
cs.plot_data_vs_ack_iter(
    200,
    filenames,
    'avg_seeds',
    'rmse',
    test_rmse_fn,
    20,
    arrs,
    to_eval,
    legend_loc=3,
)